In [ ]:
import sys
sys.path.append('..')

from scripts.data_loaders.common import show_sample
from scripts.data_loaders.CORAAL import CORAALDataset
from scripts.core.audio import TARGET_SAMPLE_RATE
from scripts.ipa_transcription.wav2vec2 import transcribe_batch

import torch
from transformers import AutoProcessor, AutoModelForCTC

from IPython.display import clear_output
clear_output()

In [2]:
data = CORAALDataset(include_speaker_info=True)

In [ ]:
aks_examples = list(data.search_transcript(" aks "))
print(f"Found {len(aks_examples)} examples of 'aks' in the transcripts")
aks_example1 = aks_examples[0]
print(aks_example1[-1])

In [ ]:
start = int(aks_example1[2]['StTime'] * TARGET_SAMPLE_RATE)
end = int(aks_example1[2]['EnTime'] * TARGET_SAMPLE_RATE)
text, audio, speaker = data[aks_example1[0]] # type: ignore
sample = (aks_example1[2]['Content'], audio[start:end], speaker) # type: ignore
show_sample(sample)

In [24]:
MODEL_ID = "facebook/wav2vec2-lv-60-espeak-cv-ft"
DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
torch.mps.empty_cache()
processor = AutoProcessor.from_pretrained(MODEL_ID)
model = AutoModelForCTC.from_pretrained(MODEL_ID).to(DEVICE)

In [ ]:
transcribe_batch([sample], model, processor)